In [1]:
#得到最具代表性的正面句子和負面句子
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [30]:
#Load Data
df = pd.read_csv('classify_reviews/food_topic.csv')
food_list = df['0'].values.tolist()

df = pd.read_csv('classify_reviews/service_topic.csv')
service_list = df['0'].values.tolist()

df = pd.read_csv('classify_reviews/ambience_topic.csv')
ambience_list = df['0'].values.tolist()

df = pd.read_csv('classify_reviews/value_topic.csv')
value_list = df['0'].values.tolist()

df = pd.read_csv('classify_reviews/others_topic.csv')
others_list = df['0'].values.tolist()

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [5]:
# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english') 
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [6]:
def TextRank(topic_list):
    sentences = []
    for x in topic_list:
        df = pd.read_csv('original_review_sets/'+x+'.txt', sep='@',header=None,quoting=3)#sep要有隨便東西 不然預設逗號間隔
        df = df.dropna(axis=1)
        sentences.append(df.values.tolist()[0])
    sentences = [y for x in sentences for y in x] # flatten list
    #sentences[:5]
    
    #Extract word vectors
    word_embeddings = {}
    f = open('data/glove.6B.100d.txt',encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    
    #Text Preprocessing
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    #句子的特徵向量
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    #相似矩陣準備 similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    
    # initialize the matrix with cosine similarity scores
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    #應用PageRank算法
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #排名
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # Extract top 5 sentences as the summary
    temp=0
    print("Most representative positive sentence:")
    for i in range(len(sentences)):
        if(sentiment_analyzer_scores(ranked_sentences[i][1])>=0.5 and len(ranked_sentences[i][1])<200):
            print(ranked_sentences[i][1]+"\n")
            temp+=1
            if(temp==5):
                break;
            continue;
    temp=0
    print("Most representative negative sentence:")
    for i in range(len(sentences)):
        if(sentiment_analyzer_scores(ranked_sentences[i][1])<=-0.25 and len(ranked_sentences[i][1])<200):
            print(ranked_sentences[i][1]+"\n")
            temp+=1
            if(temp==5):
                break;
            continue;

In [31]:
TextRank(food_list)

Most representative positive sentence:
Well, if we had the tasting menu on their website I'm certain my wife would have been a lot happier with the lobster and pork dishes especially.

We went to the Ledbury for a special occasion & as always the cuisine was superb, we had the tasting menu, coupled with the wine menu & it just took the meal to another level.

The wine was well chosen and delicious and we had a stunning dessert wine with some lovely cheese at the end.

We had the 8 courses tasting menu and it felt like the food kept on coming and the potions were all very generous.

Our Wedding Breakfast was stunning, each year the menu, the food, the flavours, the presentation gets better.

Most representative negative sentence:
There was nothing wrong with the dish but it was just not so elegant and delicate as the other dishes.

There wasn’t a poor dish in the entire menu.

The canapes were all too greasy, which was not a good way to start the evening.

I'm not saying it wasn't good 

In [32]:
TextRank(service_list)

Most representative positive sentence:
Even if extremely busy, they made our dining experience feel very special and also kindly permitted us a tour of the kitchen.

The food was superb, we did the 8 course tasting menu and indeed were lucky enough to be given an extra course to try also.

The service was good but for a restaurant like this could be better.

Well the good news for us & all future diners, is that standards of the cuisine is truly exceptional, the added bonus is the service that goes with the cuisine, this was also top draw service.

Every member of staff was friendly and went out of their way to make this dining experience a fantastic one.

Most representative negative sentence:
I'm not saying it wasn't good food; it simply wasn't great.

Maybe because of that they did not prepare a card of us beforehand (its a small matter but its their little gesture for their guests and it was unfortunate that they were not well prepared).

The staff are without exception all excelle

In [33]:
TextRank(ambience_list)

Most representative positive sentence:
I had read good reviews, and had high hopes, but nothing had prepared me, or my lovely entourage, for the stunning dining experience that we were about to enjoy.

The Ledbury offers diners a truly lovely experience; from the beautiful contemporary decor to the gorgeously plated food and escellent service, it is entirely a fine dining event.

Creative and refined cooking, great wine list, spotless service, very friendly staff, nice ambience.

Over all this was still a wonderful dining experience!

From the wonderful staff, to the relaxed and totally unstuffy atmosphere, to the exquisite food, so beautifully presented, dining here was a joy.

Most representative negative sentence:
The food styling and plating again were next to none, beautiful and artsy.

There was nothing wrong with the dish but it was just not so elegant and delicate as the other dishes.

The flavors just missed a bit.

A bit stuffy unfortunately.



In [34]:
TextRank(value_list)

Most representative positive sentence:
The food was excellent, though every course seems rather ‘structured’ that I felt less of innovation/wow-factor.

Food awesome Place great Service fantastic Overall we enjoyed everything, a bit pricy but its worth it !

The food was wonderful and the service even better.

In particular the service is very, very, good, something I am appreciating more and more these days.

I could name few different restaurants that offer great food and better value for money.

Most representative negative sentence:
If you are going to eat anywhere and you are looking for unique cuisine and exceptional service then look no further.

Fundamentally we found nothing wrong with the meals, presentation was generally very good but there was nothing particularly delicious or extraordinary on the flavour either.

Food and wine were exceptional, there is no doubt that Brett Graham would provide nothing below an exceptional level of quality.

A bit disappointing in terms of 

In [35]:
TextRank(others_list)

Most representative positive sentence:
Even if extremely busy, they made our dining experience feel very special and also kindly permitted us a tour of the kitchen.

Every member of staff was friendly and went out of their way to make this dining experience a fantastic one.

There is very little to add, except to say this was one of the best - and most memorable - meals we have had.

Very pleasant, stylish, comfortable place to sit, but fairly average food which did not come close to experiences of good one star restaurants.

Every time we visit the service is fabulous, we are always made to feel welcome.

Most representative negative sentence:
If this is not in the top world restaurants something is very wrong with the people making the decisions.

I'm not saying it wasn't good food; it simply wasn't great.

From the moment we walked in until the moment we were shown into a taxi to go home, we could not have been better looked after.

At no time were we left waiting for the next cours